In [ ]:
import glob
import json
import numpy as np
def obtain_synth_in_size(path, area_ratio=0.2):
    from glob import glob
    import json
    valid = []
    for p_label, p_png in zip(
            sorted(glob(path + '/label/*.json')),
            sorted(glob(path + '/img/*.png'))):
        with open(p_label) as f:
            j = json.load(f)['bbox']
            area = 0
            for k in j:
                v = j[k]
                # for i in range(4):
                #     v[i] = min(320 if i%2==0 else 240, max(0, v[i]))
                area = max(area, (v[2]-v[0]) * (v[3]-v[1]) / 320 / 240)
            if area < area_ratio:
                valid.append(p_png)
    return valid

from PIL import Image, ImageDraw
from IPython.display import display
import numpy as np
def draw_arm(fn):
    j = fn.replace('img', 'label').replace('png', 'json')
    m = fn.replace('img', 'mask')
    with open(j) as f:
        j = json.load(f)
    im = Image.open(m).convert('RGB')
    draw = ImageDraw.Draw(im)
    for h, f in j['keypoint'].items():
        plam = np.array(f['palm_middle.head'][:2])
        fs = np.array([p for _, p in f.items()])
        std = np.std(fs, axis=0)[:2]
        std = (std[0]**2 + std[1]**2) ** 0.5
        fs = fs.mean(axis=0)[:2]
        for i in range(8):
            p1 = plam - (plam-fs) * 0.5
            p2 = plam + (plam-fs) * 20
            p3 = plam + (plam-fs) * 3
            p1, p2 = p1.tolist(), p2.tolist()
            draw.line(p1 + p2, fill=(0, 0, 255), width=int(std*1.2))
            fs[1] -= (abs(plam[1]-fs[1])+5) * 0.05
    del draw
    mm = np.array(im)
    mm2 = np.array(Image.open(m).convert('RGB'))
    mm2[...,2] = mm[...,2]
    mm2[...,2][mm2[...,0]>0] = 0
    mm2[...,2][mm2[...,1]>0] = 0
    return Image.fromarray(mm2)


In [ ]:
path_A = ['data/DeepQ-Synth-Hand-0%d/data/s00%d' % (i, j) for i, j in
           [(1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9)]]
path_A = sum([obtain_synth_in_size(p, 0.12) for p in path_A], [])

In [ ]:
for p in path_A[:5]:
    display(draw_arm(p))
    

In [ ]:
import pathlib
from os.path import split
import tqdm
def new_path(p):
    return '/'.join([n if n[:5]!='DeepQ' else 'DeepQ-Mask-Arm' for n in p.split('/')])

for i, p in tqdm.tqdm(enumerate(path_A), total=len(path_A)):
    im = draw_arm(p)
    p = p.replace('img', 'mask')
    pathlib.Path(split(new_path(p))[0]).mkdir(parents=True, exist_ok=True)
    im.save(new_path(p))
